# Experiment: _Relabel graphs_

Here, we relabel graph nodes which occur only non-frequent in the dataset. Eg. label 'something' occurs only once in the training set, so relabel with another label that is similar to it and occurs more often.

In [ ]:
from notebook_prelude import *

In [ ]:
df = results_helper.get_experiments_by_names(['experiment_relabel_{}'.format(x) for x in [
    '5',
#    '7',
    '9',
#    '99'
]], fetch_predictions=True)

df_ = results_helper.get_results(filter_out_non_complete_datasets=None, fetch_predictions=True)
df = df.append(df_[(df_.filename.str.contains('__graph__')) & (df_.combined == False) & (df_.type == TYPE_CONCEPT_MAP)]).fillna('-').reset_index()
df = results_helper.filter_out_datasets(df, lambda x: 'RelabelGraphsTransformer' in x.graph_preprocessing.values)
print('# Results: {}'.format(len(df)))

In [ ]:
df.columns

In [ ]:
RELABELED = True
df[(df.filename.str.contains('relabeled') == False) & (df.dataset == 'ng20')][['filename', 'params']].head(1)

In [ ]:
#'graph_preprocessing__threshold',
attr = 'prediction_score_f1_macro'
group_attr = 'graph_preprocessing__threshold'
#attr = 'mean_test_f1_macro'
#df = df[df[attr] != '-']
d = df[(np.isin(df.type, (TYPE_CONCEPT_MAP))) & (df.kernel == 'wl')].groupby(['dataset',   group_attr])[attr].max().to_frame().unstack()#.reset_index()
#d_ = d[['mean_test_f1_macro']].mean_test_f1_macro.reset_index().set_index('dataset')
#d_['ratio_relabeled_vs_plain'] = (d_['RelabelGraphsTransformer'] - d_['-'])
#d_
d

In [ ]:
df.columns

In [ ]:
from utils import significance_test_utils
attr_ = 'graph_preprocessing'
confidences = collections.defaultdict(list)
for dataset, df_ in df.groupby('dataset'):
    assert group_attr in df_.columns
    print(dataset)
    best_ = df_.loc[df_.groupby(attr_)[attr].idxmax()]
    if len(best_) != 2:
        print('\tNot enough data. Skipping')
        continue
    prediction_filenames = [best_.loc[best_[attr_] == name].iloc[0].prediction_file for name in [ 'RelabelGraphsTransformer', '-']]
    diffs, score_a, score_b, global_difference, confidence = results_helper.calculate_significance(prediction_filenames[0], prediction_filenames[1])
    
    for k, v in [('Score A', score_a), ('Score B', score_b), ('Difference', global_difference), ('Confidence', confidence)]:
        print('\t{:20} {:9.4f}'.format(k, v))
    print()
    confidences['dataset'].append(dataset)
    confidences['confidence'].append(confidence)
    d.loc[dataset, 'confidence'] = confidence

In [ ]:
print(d.to_latex(float_format='%.3f'))
d